# Convert ROI to PNG


In [1]:
import cv2
import imageio.v3 as iio
import numpy as np
from read_roi import read_roi_file
import os
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from shapely.geometry import *

import pydicom
from PIL import Image
import numpy as np
import shutil
import glob

In [ ]:
# def rename_files(root_path):
#     # 定义需要去除的前缀列表
#     prefixes = ["LCC_", "LMLO_", "RCC_", "RMLO_"]

#     # os.walk() 用于遍历 root_path 下的所有文件夹和子文件夹
#     for dirpath, dirnames, filenames in os.walk(root_path):
#         for filename in filenames:
#             # 检查文件名是否以指定的任一前缀开始
#             for prefix in prefixes:
#                 if filename.startswith(prefix):
#                     # 构建完整的旧文件路径
#                     old_file_path = os.path.join(dirpath, filename)
#                     # 构建新的文件名和完整路径，即去除前缀后的路径
#                     new_filename = filename[len(prefix) :]
#                     new_file_path = os.path.join(dirpath, new_filename)
#                     # 重命名文件
#                     os.rename(old_file_path, new_file_path)
#                     print(f"Renamed '{old_file_path}' to '{new_file_path}'")
#                     break  # 匹配到一个前缀后，不需要检查其他前缀


# # 示例：给定路径
# root_path = "./dataset/inter-observer/50_Sohrab_Fati/output"  # 替换为实际的目录路径
# rename_files(root_path)

# root_path = "./dataset/inter-observer/332_Bahman_Fati/output"  # 替换为实际的目录路径
# rename_files(root_path)

In [ ]:
# import os
# import glob


# def delete_files_with_prefix(root_path, prefix):
#     """
#     删除给定路径及其所有子文件夹中具有指定前缀的文件。

#     参数:
#     - root_path: 要遍历的根目录路径
#     - prefix: 需要删除的文件前缀
#     """
#     # 遍历 root_path 下的所有文件和文件夹
#     for dirpath, dirnames, filenames in os.walk(root_path):
#         # 对于每一个文件名，检查是否符合指定的前缀
#         for filename in filenames:
#             if filename == prefix:
#                 # 构建完整的文件路径
#                 file_path = os.path.join(dirpath, filename)
#                 # 删除文件
#                 os.remove(file_path)
#                 print(f"Deleted: {file_path}")


# # 使用示例
# root_path = "./dataset/inter-observer/50_Sohrab_Fati/output"  # 替换为你的目录路径
# prefix = "1.png"  # 替换为你需要删除文件的前缀
# delete_files_with_prefix(root_path, prefix)

In [6]:
def convert_dcm_to_png(dcm_path, png_path):
    dcm_data = pydicom.dcmread(dcm_path)

    width = dcm_data.Columns
    height = dcm_data.Rows

    png_img = Image.new("L", (width, height), 0)
    png_img.save(png_path)


def process_folder(base_path):
    for folder_name in os.listdir(base_path):
        current_path = os.path.join(base_path, folder_name)
        if os.path.isdir(current_path):
            fati_path = os.path.join(current_path, "fati")
            bahman_path = os.path.join(current_path, "bahman")
            source_path = os.path.join(current_path, "source")
            if (
                os.path.exists(fati_path)
                and os.path.exists(bahman_path)
                and os.path.exists(source_path)
            ):
                for dcm_file in os.listdir(source_path):
                    if dcm_file.endswith(".dcm"):
                        # 构造 PNG 文件的完整路径
                        png_file = os.path.splitext(dcm_file)[0] + ".png"
                        png_path = os.path.join(current_path, png_file)
                        dcm_path = os.path.join(source_path, dcm_file)
                        # 转换 DICOM 到 PNG
                        dcm_data = pydicom.dcmread(dcm_path)

                        width = dcm_data.Columns
                        height = dcm_data.Rows

                        png_img = Image.new("L", (width, height), 0)
                        # png_img.save(png_img)
                        png_img.save(fati_path + "/" + png_file)
                        png_img.save(bahman_path + "/" + png_file)


# 示例：给定路径
base_path = "./dataset/inter-observer/332_Bahman_Fati/output"  # 替换为实际的路径
process_folder(base_path)

In [5]:
def convert_roi_to_png(curr_dir):
    for foldername, subfolders, filenames in os.walk(curr_dir):
        for filename in filenames:
            if filename.endswith("roi"):
                UID = os.path.splitext(filename)[0]
                png_file_name = UID + ".png"
                # print(png_file_name)
                png_path = os.path.join(foldername, png_file_name)
                roi_path = os.path.join(foldername, filename)
                if not os.path.isfile(png_path):
                    print("Not found: " + png_path)
                else:
                    # read png
                    lbl = iio.imread(png_path)
                    lbl = np.zeros(lbl.shape)
                    # read roi
                    roi = read_roi_file(roi_path)[UID]
                    if "paths" in roi.keys():
                        # read "paths"
                        paths = roi["paths"]

                        prev_polys = []
                        for ps in paths:
                            pts = np.array(ps, np.int32)

                            # prev_pt = pts[len(pts)-1]
                            # for (i,pt) in enumerate(pts):
                            #     cv2.line(lbl, prev_pt, pt, 50+205*i/len(pts), 2)
                            #     prev_pt = pt

                            cv2.fillPoly(lbl, [pts], 255)
                            curr_poly = Polygon(ps)
                            curr_area = curr_poly.area
                            for i, prev_poly in enumerate(prev_polys):
                                if prev_poly.intersects(curr_poly):
                                    prev_area = prev_poly.area
                                    common_area = prev_poly.intersection(curr_poly).area
                                    if common_area == prev_area:
                                        prev_pts = np.array(
                                            prev_poly.exterior.coords, np.int32
                                        )
                                        cv2.fillPoly(lbl, [prev_pts], 0)
                                    if common_area == curr_area:
                                        cv2.fillPoly(lbl, [pts], 0)
                                        break
                            prev_polys.append(curr_poly)
                        cv2.imwrite(png_path, lbl)
                    else:
                        # read "x" and "y"
                        x = roi["x"]
                        y = roi["y"]
                        pts = np.array(list(zip(x, y)), np.int32)
                        cv2.fillPoly(lbl, [pts], 255)  # 255 stands for white color
                        cv2.imwrite(png_path, lbl)


path = "./dataset/inter-observer/332_Bahman_Fati/output"

convert_roi_to_png(path)

Not found: ./dataset/inter-observer/332_Bahman_Fati/output/SID100735_9999.171555597976930491892230917991393644363/bahman/1.2.840.113681.2206989178.29090.3612611667.297.png
Not found: ./dataset/inter-observer/332_Bahman_Fati/output/SID100735_9999.171555597976930491892230917991393644363/bahman/1.2.840.113681.2206989178.29090.3612611667.299.png
Not found: ./dataset/inter-observer/332_Bahman_Fati/output/SID100735_9999.171555597976930491892230917991393644363/bahman/1.2.840.113681.2206989178.29090.3612611667.303.png
Not found: ./dataset/inter-observer/332_Bahman_Fati/output/SID100735_9999.171555597976930491892230917991393644363/bahman/1.2.840.113681.2206989178.29090.3612611667.301.png
Not found: ./dataset/inter-observer/332_Bahman_Fati/output/SID100735_9999.171555597976930491892230917991393644363/fati/1.2.840.113681.2206989178.29090.3612611667.297.png
Not found: ./dataset/inter-observer/332_Bahman_Fati/output/SID100735_9999.171555597976930491892230917991393644363/fati/1.2.840.113681.22069891